In [1]:
import pandas as pd
import json
import pickle
import joblib

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sys 
sys.path.append('../')
from fast_tsne.fast_tsne import fast_tsne

In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
path='canarex_coref_text/topN'
# path='canarex_coref_text_no_split_sentences/topN'
ctype='coref_text'

# path='canarex_true_text/topN'
# path='canarex_true_text_no_split_sentences/topN'
# path='relatio_test/clust_40/topN'
# ctype='true_text'

cpath = '{}/narratives_kmeans2_topN.pkl'.format(path)
narratives_path='{}/final_narratives.jsonl'.format(path)
result_path = '{}/results.jsonl'.format(path)
eval_path = '{}/eval_output.csv'.format(path)



### Load scores

In [5]:
topN = pd.read_pickle(cpath)

In [6]:
topN[topN.score == topN.score.max()]

,topN,k,score,dist,labels
0,1,4,0.263655,"[0.93414754, 0.26364124, 0.23640776, 1.0536901...","[3, 2, 2, 3, 2, 1, 2, 1, 2, 2, 1, 3, 0, 0, 1, ..."


In [7]:
# get the top N sentences and K based on score
N = topN[topN.score == topN.score.max()].iloc[0].topN
k = topN[topN.score == topN.score.max()].iloc[0].k

In [9]:
N

1

In [8]:
topN = topN[topN.score == topN.score.max()]
topN = topN.explode(['dist', 'labels']).reset_index(drop = True)

### Load document level clustering

In [9]:
k2 = pd.read_pickle('{}/kmeans2.pkl'.format(path))
k2.drop(columns='text').head(2)

,id,verb,entity1,entity2,combined,ARGM-LOC,ARGM-TMP,narrative,sentence_id,sent,sent_id,time,synthetic_label,seq,embeddings,labels_2,dist_2,score_2,noise
0,0002060a-61bb-40f6-a843-40d1acbaf858,stole,A person,over $ 1000,"[A person, over $ 1000, for]",,,[A person] (stole) [over $ 1000],0002060a-61bb-40f6-a843-40d1acbaf858_0,A person stole over $ 1000,0,295,2,0,"[-0.09445274, 0.11801259, -0.0017595241, -0.03...",0,0.0,0.0153,True
1,0002060a-61bb-40f6-a843-40d1acbaf858,delivered,the Red Cross,parcels,"[the Red Cross, parcels, to Adelaide]",at various locations,,[the Red Cross] (delivered) [parcels],0002060a-61bb-40f6-a843-40d1acbaf858_1,the Red Cross delivered parcels,1,295,2,1,"[-0.023951985, -0.03448499, -0.03293211, 0.020...",1,0.470833,0.0153,True


In [10]:
topn = k2.groupby(['id', 'labels_2']).size().reset_index(name='cl')
topn.loc[:, 'maxl'] = topn.groupby('id')['cl'].transform(max)
topn = topn[topn.cl == topn.maxl].drop(columns='maxl')
topn = pd.merge(k2, topn.drop(columns='cl'))

# get the number of narratives b/w 1-5 based on max value
if N == 1:
    print('N = 1')
    topn = topn.sort_values(['id', 'labels_2', 'dist_2'], ascending=True).groupby('id').head(1)
else:
    topn = topn.sort_values(['id', 'labels_2', 'dist_2'], ascending=True).groupby(['id', 'narrative']).head(1)
    topn = topn.sort_values(['id', 'labels_2', 'dist_2'], ascending=True).groupby('id').head(N)

topn = topn.reset_index(drop = True)

N = 1


In [11]:
topn.shape

(11110, 20)

In [12]:
topN.shape

(11110, 5)

In [13]:
topn = pd.concat([topn, topN], axis = 1)

In [14]:
topn.shape

(11110, 25)

In [15]:
topn.index

RangeIndex(start=0, stop=11110, step=1)

In [16]:
topn.columns

Index(['id', 'text', 'verb', 'entity1', 'entity2', 'combined', 'ARGM-LOC',
       'ARGM-TMP', 'narrative', 'sentence_id', 'sent', 'sent_id', 'time',
       'synthetic_label', 'seq', 'embeddings', 'labels_2', 'dist_2', 'score_2',
       'noise', 'topN', 'k', 'score', 'dist', 'labels'],
      dtype='object')

In [ ]:
topn.to_json(result_path, lines = True, orient='records')

### Calculate MSE

In [17]:
from eval_canarex import calculate_mse

In [18]:
df = pd.read_json('{}/synthetic_test_data_{}.jsonl'.format('synthetic_data', ctype), 
                  lines = True, orient='records')

In [19]:
final = pd.read_json(result_path, lines = True, orient='records')

In [20]:
eval_path

'canarex_coref_text/topN/eval_output.csv'

In [21]:
eval_df = calculate_mse(df, final)

In [22]:
eval_df[eval_df.mse == eval_df.groupby(['orig_clust'])['mse'].transform('min')]

,orig_clust,pred_clust,intercept,coef,mse,r_sq
2,1,1,-5.318633,28.490656,782.431038,0.968371
7,2,0,2.662701,30.320734,1530.570495,0.969473
9,3,2,-15.503688,28.259219,2359.21485,0.72846


In [41]:
eval_df.to_csv(eval_path, index = False)